# Setting Pyspark in Colab

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import findspark
findspark.init("/content/spark-3.2.4-bin-hadoop3.2")

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc

# RDD Exercise - Word Count (gutenberg)

1. gutenberg.txt 파일 로드
2. .map() 을 사용하여 주어진 데이터의 문장부호를 제거하고 소문자로 변환  (python 함수 .replace(), .lower() )
3. .flatmap() 을 사용하여 주어진 데이터를 단어 단위로 구분
4. .map() 을 사용하여 주어진 데이터로 (word, 1) tuple 생성
5. .reduceByKey() 를 사용하여 word를 기준으로 결합
6. 상위 10개의 결과 확인

In [ ]:
wordcount = sc.textFile("gutenberg.txt") \
              .map(lambda x: x.replace(",", "").replace(".", "").replace("'", "").replace('"', "").lower()) \
              .flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(lambda x, y: x + y) \
              .map(lambda x: (x[1], x[0])) \
              .sortByKey(False)

print(wordcount.take(10))

## 각 단계의 처리 결과 확인

In [ ]:
rdd1 = sc.textFile("gutenberg.txt")  # 파일로부터 rdd 만들기
rdd1.take(10)

In [ ]:
rdd2 = rdd1.map(lambda x: x.replace(",", "").replace(".", "").replace("'", "").replace('"', "").lower())  # 문장부호 제거 및 소문자로
rdd2.take(10)

In [ ]:
rdd3 = rdd2.flatMap(lambda x: x.split())  # 단어 단위 분리 (map이 아닌 flatMap을 사용한다는 점에 주목)
rdd3.take(10)

In [ ]:
rdd4 = rdd3.map(lambda x: (x, 1))  # reduceByKey를 사용하기 위해 (key, value)의 튜플 형태로 만듦. key=단어, value=1
rdd4.take(10)

In [ ]:
rdd5 = rdd4.reduceByKey(lambda x, y: x + y)  # reduceByKey를 사용하여 같은 key(단어)의 value(1)들을 모두 더함. 즉, 단어들의 등장 횟수를 구함
rdd5.take(10)

In [ ]:
rdd6 = rdd5.map(lambda x: (x[1], x[0]))  # sortByKey를 사용하기 위해 (key, value)의 튜플 형태로 만듦. key=단어 등장 빈도, value=단어
rdd6.take(10)

In [ ]:
rdd7 = rdd6.sortByKey(False)  # sortByKey를 사용하여 key 크기 순으로 정렬. 인자로 False를 주면 내림차순 정렬, 인자로 True를 주면 오름차순 정렬.
rdd7.take(10)  # 최빈 단어 10개 출력

# Spark DataFrame Exercise - Titanic Dataset

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# 데이터프레임 생성
titanic_df = spark.read.option("header", "true") \
                  .option("nullValue", "?") \
                  .option("inferSchema", "true") \
                  .csv('/gdrive/MyDrive/titanic_train.csv')
print('titanic 변수 type:',type(titanic_df))

In [ ]:
# 데이터프레임 자료형 보기 1
titanic_df

In [ ]:
# 데이터프레임 자료형 보기 2
titanic_df.printSchema()

In [ ]:
# 데이터프레임 자료형 보기 3
titanic_df.dtypes

In [ ]:
# 데이터프레임 크기 보기
print('DataFrame 크기: ', titanic_df.count(),'rows, ', len(titanic_df.columns),'columns')

In [ ]:
# 데이터프레임 내용 보기
titanic_df.show(3)

In [ ]:
# 데이터프레임 요약 보기
titanic_df.describe().show()

In [ ]:
# 데이터프레임 열 선택 1 : subset
titanic_df["PassengerId"].dtypes

In [ ]:
# 데이터프레임 열 선택 2 : attribute
titanic_df.PassengerId.dtypes

In [ ]:
# 데이터프레임 열 선택 3 : select
titanic_pclass = titanic_df.select("Pclass")
print(type(titanic_pclass)) # 데이터 타입이 Column이 아닌 DataFrame임에 주목
titanic_pclass.show(5)

In [ ]:
# 데이터프레임 열 생성 1 : lit() 이용
from pyspark.sql.functions import lit
titanic_df = titanic_df.withColumn("Age_0", lit(0))
titanic_df.show(3)

# withColumn(x, y) : x=Column 이름, y=Column 객체
# lit() : 전달된 값으로 채워진 Column 객체를 반환
# ex) lit(0) : 0으로 채워진 Column 객체를 반환
# 오직 리터럴(literal)만 입력으로 받을 수 있다
# 리터럴의 예) 숫자(1, 2, 3), 문자열("asdf", "hello") 등
# 불가능한 입력의 예) 리스트([1, 2]), 튜플((1, 2)) 등

In [ ]:
# 데이터프레임 열 생성 2 : 다른 Column으로부터
from pyspark.sql.functions import col
titanic_df = titanic_df.withColumn("Age_by_10", col("Age") * 10)
titanic_df = titanic_df.withColumn("Family_No", col("SibSp") + col("Parch") + 1)
titanic_df.show(3)

# col() : 전달된 값의 이름을 가진 Column 객체를 반환

In [ ]:
# 데이터프레임 열 수정
from pyspark.sql.functions import col
titanic_df = titanic_df.withColumn("Age_by_10", col("Age_by_10") + 100)  # 덮어씌워짐
titanic_df.show(3)

In [ ]:
# 데이터프레임 열 삭제
titanic_df = titanic_df.drop("Age_0")
titanic_df.show(3)

In [ ]:
# drop 명령어는 원본을 수정하지 않음
drop_result = titanic_df.drop('Age_0', 'Age_by_10', 'Family_No')
print('drop 후 반환된 값:', type(drop_result))
titanic_df.show(3)
drop_result.show(3)

titanic_df = drop_result

In [ ]:
# 데이터프레임 조건부 선택 : filter, where
titanic_df.where(titanic_df.Pclass == 3).show(3)
titanic_df.where(titanic_df['Pclass'] == 3).show(3)
titanic_df.filter(titanic_df.Pclass == 3).show(3)
titanic_df.filter(titanic_df['Pclass'] == 3).show(3)

In [ ]:
# 데이터프레임 조건부 선택 : Boolean Indexing
titanic_boolean = titanic_df.filter(titanic_df['Age'] > 60).show(3)

titanic_df.filter(titanic_df['Age'] > 60).select('Name','Age').show(3)

titanic_df.filter((titanic_df['Age'] > 60) & (titanic_df['Pclass'] == 1) & (titanic_df['Sex'] == 'female')).show()

cond1 = titanic_df['Age'] > 60
cond2 = titanic_df['Pclass'] == 1
cond3 = titanic_df['Sex'] == 'female'
titanic_df.filter(cond1 & cond2 & cond3).show()

In [ ]:
# 데이터프레임 정렬
titanic_df.sort('Name').show(3)

titanic_df.sort(['Pclass', "Name"], ascending=[False, False]).show(3)

In [ ]:
# 데이터프레임 묶기(groupby)
titanic_groupby = titanic_df.groupby('Pclass')
titanic_groupby.mean().show()

titanic_df.groupBy("Pclass").count().show()

In [ ]:
# Groupby Aggregation
from pyspark.sql.functions import max, min, sum, avg, count

titanic_df.groupby('Pclass').agg(max('Age'), min('Age')).sort('Pclass').show()

exprs = [count('Age'), sum('SibSp'), avg('Fare')]
titanic_df.groupby('Pclass').agg(*exprs).sort('Pclass').show()

exprs = [count(x).alias(x) for x in titanic_df.columns]
titanic_groupby = titanic_df.groupby('Pclass').agg(*exprs)
titanic_groupby.show()
# NULL 값은 세어지지 않음
# cf) titanic_df.filter(titanic_df["Pclass"] == 1).filter("Age is not NULL").count() == 186
# cf) titanic_df.filter(titanic_df["Pclass"] == 3).filter("Cabin is not NULL").count() == 12

In [ ]:
# Aggregation
titanic_df.agg(count(col('PassengerId')), count(col('Age')), count(col('Cabin'))).show()

exprs = [count(x) for x in titanic_df.columns]
titanic_df.agg(*exprs).show()

titanic_df.agg(avg(col('Age')),avg(col('Fare'))).show()

In [ ]:
# 결손 데이터
from pyspark.sql.functions import when

exprs = [(col(x).isNull()).alias(x) for x in titanic_df.columns]
titanic_df.select(*exprs).show(3)

titanic_df.select([count(when(col(x).isNull(), x)).alias(x) for x in titanic_df.columns]).show(3)

In [ ]:
# 결손 데이터 채우기
titanic_df = titanic_df.fillna({'Cabin':'C000'})
titanic_df.show(3)

Age_mean = titanic_df.agg(avg('Age').alias('Age')).first().asDict()
print(Age_mean)
titanic_df = titanic_df.fillna(Age_mean) # 평균값으로 채우기
titanic_df = titanic_df.fillna({'Embarked':'S'})
titanic_df.select([count(when(col(x).isNull(), x)).alias(x) for x in titanic_df.columns]).show(3)

# 기타

In [ ]:
# join
df1 = sc.parallelize([["Michael", None], ["Andy", 30], ["Justin", 19]]).toDF(['name', 'age'])
df2 = sc.parallelize([["Michael", 2000], ["Andy", 2500], ["Justin", 5000], ["Bob", 6500]]).toDF(['name', 'income'])

df3 = df1.join(df2, on="name", how="inner")
df3.show()

df4 = df1.join(df2, on="name", how="outer")
df4.show()

In [ ]:
# SQL
df = sc.parallelize([["Michael", None], ["Andy", 30], ["Justin", 19]]).toDF(['name', 'age'])

df.createOrReplaceTempView("people")
sqlDF = spark.sql("SELECT * from people")
sqlDF.show()

## Lambda 함수

In [ ]:
def get_square(a):
    return a**2
print('3의 제곱은:',get_square(3))

In [ ]:
lambda_square = lambda x : x**2
print('3의 제곱은:',lambda_square(3))

In [ ]:
a=[1,2,3]
square = map(lambda x: x**2, a)
list(square)

## udf

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

slen = udf(lambda s: len(s), IntegerType())

@udf
def to_upper(s):
  if s is not None:
    return s.upper()

@udf(returnType=IntegerType())
def add_one(x):
  if x is not None:
    return x + 1

df = spark.createDataFrame([(1, "John Doe", 21)], ("id", "name", "age"))
df.select(slen("name").alias("length of name"), to_upper("name"), add_one("age")).show()

In [ ]:
# CHild Adult 열 생성
from pyspark.sql.functions import udf
from pyspark.sql.types import *

get_age_cat = udf(lambda x: 'Child' if x<=15 else 'Adult', StringType())
titanic_df = titanic_df.withColumn('Child_Adult', get_age_cat(titanic_df['Age']))
titanic_df.select('Age','Child_Adult').show(8)

In [ ]:
# Age Category 열 생성
lambda_age_cat = lambda x: 'Child' if x<=15 \
                                  else ('Adult' if x <= 60 else 'Elderly')
get_age_cat = udf(lambda_age_cat, StringType())
titanic_df = titanic_df.withColumn('Age_cat', get_age_cat(titanic_df['Age']))
titanic_df.groupby('Age_cat').count().show()

In [ ]:
def get_category(age):
  cat = ''
  if age <= 5:
    cat = 'Baby'
  elif age <= 12:
    cat = 'Child'
  elif age <= 18:
    cat = 'Teenager'
  elif age <= 25:
    cat = 'Student'
  elif age <= 35:
    cat = 'Young Adult'
  elif age <= 50:
    cat = 'Adult'
  else:
    cat = 'Elderly'
  return cat

udf_get_category = udf(get_category, StringType())
titanic_df = titanic_df.withColumn('Age_cat', udf_get_category(titanic_df['Age']))
titanic_df.select('Age','Age_cat').show(5)

# 따릉이 데이터

In [ ]:
# 대여소 데이터 불러오기
df_loc = spark.read\
              .option("header","true")\
              .option("inferSchema","true")\
              .csv('/gdrive/MyDrive/Colab Notebooks/bikeshare_loc.csv')
df_loc.show(10)

In [ ]:
# 일간 대여/반납 데이터 불러오기
df_usage = spark.read\
                .option("header","true")\
                .option("inferSchema","true")\
                .csv('/gdrive/MyDrive/Colab Notebooks/bikeshare_seoul.csv')
df_usage.show(10)

## 따릉이 대여소 번호 2319의 정보는?

In [ ]:
result_df = "fill here"  # df_loc에서 "대여소번호" column의 값이 2319인 행을 필터

result_df.show()

## 따릉이 대여소는 어느 구에 가장 많이 있을까?

In [ ]:
result_grouped = "fill here"  # df_loc dataframe을 '구명' column으로 grouping한다

result_grouped_counted = result_grouped."fill here"  # 각 그룹별로 몇 개의 항목이 있는지 count한다.

result_grouped_counted_sorted = result_grouped_counted."fill here"  # count column의 내림차순으로 정렬

result_grouped_counted_sorted.show()

In [ ]:
# 일간 대여/반납 데이터 불러오기
df_usage = spark.read\
                .option("header","true")\
                .option("inferSchema","true")\
                .csv('/gdrive/MyDrive/Colab Notebooks/bikeshare_seoul.csv')

df_usage.show(10)

## 따릉이 대여소 번호 2319의 정보는?

In [ ]:
result_df = df_loc.filter(df_loc['대여소번호'] == 2319)  # df_loc에서 "대여소번호" column의 값이 2319인 행을 필터

result_df.show()

## 따릉이 대여소는 어느 구에 가장 많이 있을까?

In [ ]:
result_grouped = df_loc.groupBy('구명')  # df_loc dataframe을 '구명' column으로 grouping한다

result_grouped_counted = result_grouped.count()  # 각 그룹별로 몇 개의 항목이 있는지 count한다.

result_grouped_counted_sorted = result_grouped_counted.sort("count",ascending=False)  # count column의 내림차순으로 정렬

result_grouped_counted_sorted.show()

## 각 구의 따릉이 대여소 수를 Bar Graph로 그려보자

In [ ]:
!pip install -q koreanize-matplotlib

import koreanize_matplotlib

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
list_pd = df_loc.select('구명').toPandas()  # df_loc dataframe에서 "구명" column만 select한 후, pandas dataframe으로 변경한다.

sns.countplot(x='구명', data=list_pd)  # sns.countplot : data 인자를 x 인자 column으로 grouping해 각 그룹에 몇 개의 항목이 있는지 그래프를 그린다.

plt.xticks(rotation=90)  # x축 눈금들을 90도 회전해서 출력한다.

plt.show()

## 따릉이 거치대가 가장 많은 구는 어디일까?

In [ ]:
result_grouped = "fill here"  # df_loc dataframe을 '구명' column으로 grouping한다

result_grouped_sum = result_grouped."fill here"  # 같은 "구명" column을 가진 행들의 "거치대수" column들을 모두 더한다.

result_grouped_sum_sorted = result_grouped_sum."fill here"  # 덧셈 결과에 따라 내림차순 정렬한다.

result_grouped_sum_sorted.show(1)

## 각 구의 따릉이 거치대 수를 Bar Graph로 그려보자

In [ ]:
result = "fill here".toPandas()  # df_loc dataframe을 "구명" column으로 grouping하고, 같은 "구명" column을 가진 항목끼리 "거치대수" column을 모두 더한 후, pandas dataframe으로 변환한다.

sns.barplot(x="구명", y="sum(거치대수)", data=result)  # sns.barplot : data 인자의 x 인자 column과 y 인자 column 간의 막대그래프를 그린다.
plt.xticks(rotation=90)  # x축 눈금들을 90도 회전해서 출력한다.
plt.show()

## 강남구 2318번 따릉이 대여소의 2018년 5월 일간 따릉이 대여건수 추이는?

In [ ]:
from pyspark.sql.functions import col, split

U2318_may = df_usage.where("fill here")  # df_usage dataframe의 "기준일자" column이 2018년 5월 1일 ~ 2018년 5월 31일 사이에 있는 항목들만 선택한다.
U2318_may = U2318_may.select("fill here")  # "기준일자", "대여건수", "반납건수", "대여소번호" column만 선택한다. 이때 "대여소번호" column은 "대여소명" column을 "."으로 split했을 때 가장 앞에 나오는 항목을 가져와서 만든다.
U2318_may = U2318_may.filter(col('대여소번호') == 2318)  # "대여소번호" column이 2318인 항목들만 선택한다.

U2318_may.show(40)

In [ ]:
plt.figure(figsize=(16, 8))  # 그래프 크기 설정
sns.lineplot(x='기준일자', y='대여건수', data=U2318_may.toPandas())  # U2318_may dataframe을 pandas dataframe으로 변경하여 꺾은선그래프를 그린다.
plt.xticks(rotation=90)  # x축 눈금들을 90도 회전해서 출력한다.
plt.show()

## 2018년 중 일간 최고 대여건수를 기록한 대여소는?

In [ ]:
daily_best_df = df_usage."fill here"  # df_usage dataframe에서 "기준일자" column이 2018년 01월 01일 ~ 2018년 12월 31일 사이에 있는 항목을 선택한다.

daily_best_df = daily_best_df."fill here"  # "대여소위치", "대여소명", "기준일자", "대여건수" column만 선택한다.

daily_best_df = daily_best_df."fill here"  # "대여건수" column에 대해 내림차순 정렬한다.

daily_best_df.show(1)

## 2018년 중 일간 최고 반납건수를 기록한 대여소는?

In [ ]:
# "2018년 중 일간 최고 대여건수를 기록한 대여소는?" 항목을 참조해서 직접 해 봅시다.
"fill here"

## 2018년 월별 강남구의 따릉이 대여소의 월 누적 대여/반납건수의 추이는?

In [ ]:
import pyspark.sql.functions as sf

#누적 대여건수
U2018_g_d = df_usage."fill here"  # df_usage dataframe의 "대여소위치", "대여건수", "월" column만 선택한다. 이때 "월" column은 "기준일자" column으로부터 만든다("abcd-ef-gh" 형태에서 가운데 "ef" 부분만 가져온다).
U2018_g_d = U2018_g_d."fill here"  # "대여소위치" column을 이용해 grouping한다.
U2018_g_d = U2018_g_d.pivot('월').agg(sf.sum('대여건수'))  # (grouping된 column인) "대여소위치"를 행으로, (pivot() 메소드에 인자루 주어진 column인) "월"을 열로 하는 pivot table을 만들고, 각 셀의 값으로 해당 "대여소위치"와 "월"에 해당하는 항목들의 "대여건수"를 모두 더한 값을 채워 새로운 dataframe을 만든다.
U2018_g_d = U2018_g_d."fill here"  # "대여소위치" column이 "강남구"인 column들만 선택한다.
U2018_g_d.show()

#누적 반납건수 : 위 "누적 대여건수" 항목을 참조해서 직접 해 봅시다.
U2018_g_b = "fill here"
U2018_g_b.show()

In [ ]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters

U2018_g_d_pd = U2018_g_d.toPandas().transpose()[1:].reset_index()  # 만들어진 U2018_g_d dataframe을 pandas dataframe으로 바꾼다.
U2018_g_d_pd.columns = ['월', '누적대여건수']  # column명을 "월", "누적대여건수"로 바꾼다.
U2018_g_d_pd = U2018_g_d_pd.apply(pd.to_numeric)  # 각 column들을 숫자 데이터 타입으로 바꾼다.

U2018_g_b_pd = U2018_g_b.toPandas().transpose()[1:].reset_index()  # 만들어진 U2018_g_b dataframe을 pandas dataframe으로 바꾼다.
U2018_g_b_pd.columns = ['월', '누적반납건수']  # column명을 "월", "누적대여건수"로 바꾼다.
U2018_g_b_pd = U2018_g_b_pd.apply(pd.to_numeric)  # 각 column들을 숫자 데이터 타입으로 바꾼다.

In [ ]:
plt.figure(figsize=(16, 8))  # 그래프 크기 설정
sns.lineplot(x='월', y = '누적대여건수', data = U2018_g_d_pd)  # U2018_g_d_pd pandas dataframe의 꺾은선그래프를 그린다.
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))  # 그래프 크기 설정
sns.lineplot(x='월', y = '누적반납건수', data = U2018_g_b_pd)  # U2018_g_b_pd pandas dataframe의 꺾은선그래프를 그린다.
plt.show()

## 2018년 일 평균 대여건수가 가장 큰 대여소는?

In [ ]:
result = "fill here"  # df_usage dataframe의 "기준일자" column의 값이 2018년 01월 01일 ~ 2018년 12월 31일 사이에 있는 값들만 선택한다.

result = result."fill here"  # "대여소명" column으로 grouping한다.

result = result."fill here"  # "대여소명"이 같은 항목들에 대해 "대여건수" column의 평균을 구해 "평균대여건수"라는 이름의 column으로 만든다.

result = result."fill here"  # "평균대여건수" column에 대해 내림차순 정렬한다.

result.show(1)

## 2018년 누적 대여건수가 가장 많은 대여소는?

In [ ]:
result = "fill here"  # df_usage dataframe의 "기준일자" column의 값이 2018년 01월 01일 ~ 2018년 12월 31일 사이에 있는 값들만 선택한다.

result = result."fill here"  # "대여소명" column으로 grouping한다.

result = result."fill here"  # "대여소명"이 같은 항목들에 대해 "대여건수" column의 합계을 구해 "총대여건수"라는 이름의 column으로 만든다.

result = result."fill here"  # "총대여건수" column에 대해 내림차순 정렬한다.

result.show(1)

## 2018년 누적 반납건수가 가장 많은 대여소는?

In [ ]:
# 위 "2018년 누적 대여건수가 가장 많은 대여소는?" 항목을 참조해서 직접 해 봅시다.
"fill here"

## 2018년 누적 대여건수가 가장 많은 지역구는?

In [ ]:
result = "fill here"  # df_usage dataframe의 "기준일자" column의 값이 2018년 01월 01일 ~ 2018년 12월 31일 사이에 있는 값들만 선택한다.

result = result."fill here"  # "대여소위치" column으로 grouping한다.

result = result."fill here"  # "대여소위치"가 같은 항목들에 대해 "대여건수" column들의 합을 구해 "총대여건수"라는 이름의 column으로 만든다.

result = result."fill here"  # "총대여건수" column에 대해 내림차순 정렬한다.

result.show(1)

## 2018년 누적 대여건수와 반납건수의 차이가 가장 큰 대여소/지역구는?

In [ ]:
result = "fill here"  # df_usage dataframe의 "기준일자" column의 값이 2018년 01월 01일 ~ 2018년 12월 31일 사이에 있는 값들만 선택한다.

result = result."fill here"  # "대여건수" column과 "반납건수" column의 차를 계산해 "대여차이" column을 만든다.

result = result."fill here"  # "대여소위치" column으로 grouping한다.

result = result."fill here"  # "대여소위치"가 같은 gkdahremfdp eogo "대여차이" column의 합을 구해 "총대여차이"라는 이름의 column을 만든다.

result = result."fill here"  # "총대여차이" column으로 내림차순 정렬한다.

result.show(1)

## 대여건수가 100건 이상을 기록한 적이 있는 따릉이 대여소 중 가장 남쪽에 있는 대여소는?

In [ ]:
usage100 = "fill here"  # df_usage dataframe에서 "대여건수" column의 값이 100 이상인 항목들만 선택한다.

usage100 = usage100."fill here"  # "대여건수", "반납건수", "대여소번호" column을 선택한다. 이때 "대여소번호" column은 "대여소명" column을 "."으로 split했을 때 가장 앞에 나오는 항목을 가져와서 만든다.

usage100.show(10)

In [ ]:
df_loc_selected = "fill here"  # df_loc dataframe에서 "대여소번호", "구명", "대여소명", "위도" column만 선택한다.

df_joined = usage100."fill here"  # 위에서 만든 usage100 dataframe과 df_loc_selected dataframe을 "대여소번호" column에 대해 inner join한다.

df_joined = df_joined."fill here"  # "위도"에 대해 오름차순 정렬한다.

df_joined.show(1)

## 따릉이 거치대 수 대비 따릉이 대여/반납건수 비율이 가장 높은 대여소는?

In [ ]:
df_usage_selected = "fill here"  # df_usage dataframe에서 "대여소위치", "대여건수", "반납건수", "대여소번호" column을 선택한다. 이때 "대여소번호" column은 "대여소명" column을 "."으로 split했을 때 가장 앞에 나오는 항목을 가져와서 만든다.

df_loc_selected = "fill here"  # df_loc dataframe에서 "대여소번호", "거치대수", "대여소명", "위도" column을 선택한다.

df_joined = "fill here"  # df_usage_selected dataframe과 df_loc_selected dataframe을 "대여소번호" column에 대해 inner join한다. (두 테이블에 모두 존재하는 대여소만 사용하기 위함)

result = df_joined."fill here"  # "대여소명" column으로 grouping한다.

result = result."fill here"  # "대여소명" column의 값이 같은 항목들에 대해 "대여건수" column의 합을 구해 "총대여건수"라는 이름의 column을 만들고, "거치대수" column의 평균을 구해 "평균거치대수"라는 이름의 column을 만든다.

result = result."fill here"  # "총대여건수" column을 "평균거치대수" column으로 나눈 "대여비율"이라는 이름의 column을 만든다.

result = result."fill here"  # "대여비율" column에 대해 내림차순 정렬한다.

result.show(1)

# Pandas API on Spark

In [ ]:
import pandas as pd
import pyspark.pandas as ps
from IPython.display import display # for just pretty print

titanic_pd = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/titanic_train.csv')
display(titanic_pd.head(5))

titanic_ps = ps.from_pandas(titanic_pd)
display(titanic_ps.head(5))

# Is it always possible? No if the source data(titanic_train.csv) is too big to fit in local node.

In [ ]:
# So, first you need to read the source with RDD or PySpark Dataframe.
# This operation is not limited to the local memory size
titanic_df = spark.read.option("header", "true") \
                  .option("inferSchema", "true") \
                  .csv('/gdrive/MyDrive/Colab Notebooks/titanic_train.csv')

display(titanic_df.head(5))

# Then you can pass the data to Pandas API on Spark.
titanic_ps = titanic_df.to_pandas_on_spark()
display(titanic_ps.head(5))

# You can pass the data to local Pandas DataFrame, after some diet on data.
titanic_pd = titanic_ps.to_pandas()
display(titanic_pd.head(5))